In [42]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import random
from xgboost import XGBClassifier

In [97]:
train_df = pd.read_csv("train.csv")
x_train = train_df.drop(["SalePrice"], axis = 1)
y_train = train_df["SalePrice"]
x_test = pd.read_csv('test_toup.csv')

In [98]:
x_train.shape

(1321, 80)

In [99]:
x_whole = pd.concat([x_train, x_test], axis = 0, ignore_index=True)
print(x_train.shape)
print(x_test.shape)
print(x_whole.shape)

(1321, 80)
(139, 80)
(1460, 80)


In [100]:
object_dtype = []

for i in x_whole.columns:
    if x_whole[i].dtype == 'object':
        object_dtype.append(i)
        
np.array(object_dtype)

array(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence',
       'MiscFeature', 'SaleType', 'SaleCondition'], dtype='<U13')

In [101]:
x_whole['MasVnrType'].unique()

array(['None', 'BrkFace', 'Stone', 'BrkCmn', nan], dtype=object)

In [102]:
valid_na_columns = ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']

In [103]:
num_but_obj_columns = ['MSSubClass', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd','GarageCars', 'Fireplaces', 'GarageYrBlt', 'MoSold', 'YrSold']

In [104]:
#for int and float columns
meanable_cols = []


for i in x_whole.columns:
    if i not in num_but_obj_columns:
        if x_whole[i].dtypes == 'int' or x_whole[i].dtypes == 'float':
            meanable_cols.append(i)
            
meanable_cols

['Id',
 'LotFrontage',
 'LotArea',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal']

In [108]:
le = preprocessing.LabelEncoder()

for i in x_whole.columns:
    if i not in valid_na_columns:
        if i in meanable_cols:
            x_whole[i].fillna(-1, inplace = True)
            for j in range(len(train_df[i])):
                if train_df[i][j] < 0:
                    x_whole[i][j] = random.choice([x_whole[i].mean() - x_whole[i].std(), x_whole[i].mean() + x_whole[i].std()])
        else:
            print(i)
            x_whole[i].fillna("None", inplace = True)
            for j in range(len(x_whole[i])):
                if x_whole[i][j] == "None":
                    x_whole[i][j] = random.choice(x_whole[i].dropna().unique())
            if i in object_dtype:
                x_whole[i] = le.fit_transform(x_whole[i])
                
    else:
        print(i)
        x_whole[i].fillna('NaN', inplace=True)
        x_whole[i] = le.fit_transform(x_whole[i])

MSSubClass
MSZoning
Street
Alley
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
OverallQual
OverallCond
YearBuilt
YearRemodAdd
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath


/home/rohan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


BedroomAbvGr
KitchenAbvGr
KitchenQual
TotRmsAbvGrd
Functional
Fireplaces
FireplaceQu
GarageType
GarageYrBlt
GarageFinish
GarageCars
GarageQual
GarageCond
PavedDrive
PoolQC
Fence
MiscFeature
MoSold
YrSold
SaleType
SaleCondition


In [124]:
x_whole['GarageYrBlt'] = x_whole['GarageYrBlt'].astype(float)
x_whole['GarageYrBlt'].unique()

array([1997., 1922., 2005., 2000., 1999., 1963., 2004., 1931., 2001.,
       1936., 1975., 2007., 1971., 1960., 1923., 1983., 1950., 2009.,
       2006., 1984., 2003., 1928., 1926., 1969., 1955., 1981., 1991.,
       1987., 1961., 1986., 1988., 1958., 1935., 1920., 1957., 1993.,
       1990., 1916., 1979., 1932., 1972., 1989., 1976., 1918., 1982.,
       1980., 1924., 1948., 1962., 1964., 1998., 1996., 1977., 1940.,
       1939., 2002., 1995., 1974., 1959., 1949., 2008., 1994., 1910.,
       1954., 1956., 1978., 1965., 1966., 1951., 1992., 1973., 1970.,
       1953., 1968., 1929., 1925., 1941., 1930., 1947., 2010., 1985.,
       1927., 1914., 1937., 1921., 1942., 1915., 1938., 1967., 1952.,
       1934., 1906., 1945., 1946., 1908., 1933., 1900.])

In [125]:
x_whole.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,140,60,3,65.0,15426,1,1,0,3,0,...,0,0,3,2,1,0,8,2009,8,4
1,141,20,3,70.0,10500,1,1,3,3,0,...,0,0,3,4,1,0,4,2010,4,4
2,142,20,3,78.0,11645,1,1,3,3,0,...,0,0,3,4,1,0,1,2006,8,4
3,143,50,3,71.0,8520,1,1,3,3,0,...,0,0,3,2,1,0,6,2010,8,4
4,144,20,3,78.0,10335,1,1,0,3,0,...,0,0,3,4,1,0,6,2009,8,4


In [126]:
x_train = x_whole[:-139]
x_test = x_whole[:139]
print(x_train.shape)
print(x_test.shape)

(1321, 80)
(139, 80)


In [127]:
model = XGBClassifier()

In [128]:
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [129]:
y_pred = model.predict(x_test)

In [130]:
y_pred

array([231500, 115000, 260000, 166000, 204000, 125000, 130000, 105000,
       222500, 141000, 115000, 122000, 220000, 190000, 235000, 125000,
        79000, 109500, 269500, 231500, 320000, 162500, 325000, 220000,
       115000, 152000, 127500, 190000, 260000, 183500, 228000, 128500,
       215000, 239000, 163000, 184000, 155000, 211000, 172500, 501837,
       100000, 177000, 219500, 120000, 200000, 127000, 250000, 173000,
       135000, 135000, 286000, 315000, 184000, 192000, 130000, 127000,
       148500, 311872, 235000, 139000, 274900, 140000, 171500, 112000,
       149000, 110000, 180500, 143900, 141000, 277000, 145000,  98000,
       140000, 252678, 156000, 162000, 135000, 210000, 107000, 205000,
       140000, 213500, 200000, 179900,  97000, 266000, 112000, 290000,
       106000, 125000, 192500, 148000, 268000,  89500, 138500, 236000,
        89500, 192000, 194500, 318000, 113000, 262500, 110500,  79000,
       120000, 205000, 241500, 137000, 140000, 180000, 277000,  76500,
      

In [131]:
df_out = pd.DataFrame(y_pred)

In [132]:
df_out


,0
0,231500
1,115000
2,260000
3,166000
4,204000
...,...
134,139000
135,124500
136,205000
137,201000


In [135]:
df_out.columns = ["SalePrice"]

In [138]:
df_out["Id"] = df_out.index

In [142]:
df_out = df_out[["Id", "SalePrice"]]

In [143]:
df_out


,Id,SalePrice
0,0,231500
1,1,115000
2,2,260000
3,3,166000
4,4,204000
...,...,...
134,134,139000
135,135,124500
136,136,205000
137,137,201000


,SalePrice
Id,
0,231500
1,115000
2,260000
3,166000
4,204000
...,...
134,139000
135,124500
136,205000


In [147]:
df_out["Id"] = range(1, 140)

In [149]:
df_out.set_index("Id")

,SalePrice
Id,
1,231500
2,115000
3,260000
4,166000
5,204000
...,...
135,139000
136,124500
137,205000


In [152]:
df_out.to_csv("ouput.csv", index = False)